In [135]:
import pandas as pd
import numpy as np
import requests

In [136]:
kewy_list="T:\colin\BRice\V2\gwas-gene-discovery\mock_keyword_list.txt"
disc_genes="T:\colin\BRice\V2\gwas-gene-discovery\MLM.DTF.GWAS.Results\summary_genes_discovered.txt"

In [137]:
def append_summary(disc_genes):
    with open(kewy_list, "r") as fk:
        pheno=[]
        for line in fk:
            pheno.append(line.rstrip())
        print(pheno)
        summary=pd.read_csv(disc_genes, sep="\t")

        #creating knetminer genepage urls.
        network_view=[]
        keyw2 = "+OR+".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
        print(keyw2)
        for i in summary[u'GENE']:
            link="http://knetminer.rothamsted.ac.uk/{}/genepage?list={}&keyword={}".format("riceknet", i, keyw2)
            r=requests.get(link)
            network_view.append(r.url)
            #look a few days back on how to make this a new column of genetable.
        
        
        #obtaining knetscores for genes
        keyw1 = "%20OR%20".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
        genestr=(",").join(list(summary[u'GENE']))
        link="http://knetminer.rothamsted.ac.uk/riceknet/genome?"
        parameters={"keyword":keyw1, "list":genestr}
        r=requests.get(link, params=parameters)
        
        #check if requests is successful
        if not r.ok:
                r.raise_for_status()
                sys.exit()

        #extract unicode string of geneTable decoded from json
        decoded=r.json()[u'geneTable'].split("\t")
        #remove space or newline at the end
        decoded=(decoded)[:-1]
        
        colnum=9
        #tabulate genetable into 9 columns.
        genetable=np.array(decoded).reshape(len(decoded)//colnum, colnum)
        genetable=pd.DataFrame(genetable[1:,:], columns=genetable[0,:])
        
        knetgenes=list(genetable[u'ACCESSION'])
        knetscores=list(genetable[u'SCORE'])
        
        knetdict=dict(zip(knetgenes, knetscores))
        ordered_score=[]
        for i in summary[u'GENE']:
            i=i.upper()
            #print(i)
            #print(knetdict[u'{}'.format(i)])
            ordered_score.append(knetdict[u'{}'.format(i)])
        #print(ordered_score)
        summary[u'SCORE'] = ordered_score
        summary[u'network_view']=network_view

        summary.to_csv(disc_genes, sep="\t", index=False)
        #return summary

In [138]:
append_summary(disc_genes)

['Germination', 'Coleoptile length', 'Mesocotyl length']
(Germination)+OR+(Coleoptile+AND+length)+OR+(Mesocotyl+AND+length)
